In [ ]:
import collections
import random
import sys
import math
import numpy as np


# The following parameters correspond to "Case 1" in Fig8.

th=200
R_K=3  # R_K=lambda_{K,h}/lambda_{K,l}
R_L=1  # R_L=lambda_{L,h}/lambda_{L,l}
N=100
K=1000
W=10000
T=2000
P_K=0.2
P_L=0.8
n=5

inputFile='randomact_n'+str(n)+'_PK'+str(P_K)+'_PL'+str(P_L)+'_r'+str(R_K)+'_rL'+str(R_L)+'_th'+str(th)+'_N'+str(N)+'_K'+str(K)


# make the time series of K(t), L(t) and W(t)

K_tDic=collections.defaultdict(int)
L_tDic=collections.defaultdict(int)
W_tDic=collections.defaultdict(int)
f=open('Temporal_network/'+inputFile+'.txt','r')
for line in f:
    ori=line.split()
    time,node1,node2=map(int,ori)            
    if node1>node2:node1,node2=node2,node1
    link=node1,node2
    if link not in linkDic:
        degDic[node1]+=1
        degDic[node2]+=1
        K_tDic[time]+=1
    else:
        L_tDic[time]+=1
    linkDic[link]+=1
    W_tDic[time]+=1    
f.close()


Kt_List=[]
Lt_List=[]
Wt_List=[]
timeList=list(W_tDic.keys())
tmax=max(timeList)
for t in range(tmax):
    if t in timeList:
        Kt_List.append(Kt)
        Lt_List.append(Lt)
        Wt_List.append(Wt)
    else:
        Kt_List.append(0)
        Lt_List.append(0)
        Wt_List.append(0)
    
    
    
# Calculate the sample entropy of {K(t)}, {L(t)}, {W(t)} for various scale factors

def SampEn(U, m, r):
    def _maxdist(x_i, x_j):
        result = max([abs(ua - va) for ua, va in zip(x_i, x_j)])
        return result
    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) for x_i in x]
        return sum(C) - 1.0
    N = len(U)    
    return -np.log(_phi(m+1) / _phi(m))

# default: m=2, rnum=0.15

m=2
rnum=0.15
max_tw=101
KtSD=np.std(Kt_List)
LtSD=np.std(Lt_List)
WtSD=np.std(Wt_List)

r=rnum*KtSD
r2=rnum*LtSD
r3=rnum*WtSD

Kt_SDic=collections.defaultdict(float)
Lt_SDic=collections.defaultdict(float)
Wt_SDic=collections.defaultdict(float)
for tw in range(1,max_tw):
    if tw<=tmax:
        newKt_List=[]
        newLt_ist=[]
        newWt_List=[]
        check=0
        Kty=0
        Lty=0
        Wty=0
        for i in range(len(Kt_List)):
            Kty+=Kt_List[i]
            Lty+=Lt_List[i]
            Wty+=Wt_List[i]
            check+=1
            if check==tw:
                newKt_List.append(Kty/float(tw))
                newLt_List.append(Lty/float(tw))
                newWt_List.append(Wty/float(tw))
                check=0
                Kty=0
                Lty=0
                Wty=0

        KtS=SampEn(newKt_List, m, r)
        LtS=SampEn(newLt_List, m, r2)
        WtS=SampEn(newWt_List, m, r3)

        Kt_SDic[tw]=KtS
        Lt_SDic[tw]=LtS
        Wt_SDic[tw]=WtS

twList=list(Kt_SDic.keys())
twList.sort()
f=open('sample_entropy/MSE_Kt_Lt_Wt_'+inputFile+'_m_'+str(m)+'_r_'+str(rnum)+'.txt','w')
for i in twList:
    f.write(str(i)+'\t'+str(Kt_SDic[i])+'\t'+str(Lt_SDic[i])+'\t'+str(Wt_SDic[i])+'\n')
f.close()
